In [1]:
from omegaconf import OmegaConf
import numpy as np
import os


from ldm.models.diffusion.ddim import DDIMSampler
from analogy_creator import AnalogyCreator
import utils

/home/subrtade/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


# Presets


In [2]:
path_to_data = './dataset/data/'
out_subfolder = 'notebook_analogies'
experiment_root = './results'
visualize_tokens = True
# guidance scales
scales = [1.,2., 3., 5., 7., 9., 12.]
# analogy strength step
steps = np.linspace(0, 3, 20)

# Initialization

In [ ]:
config = OmegaConf.load('./config/parameter_estimation.yaml')


print('Loading model...')
model = utils.prepare_default_model()
print('Model loaded')


export_path = os.path.join(experiment_root, 'analogy_results', out_subfolder)
os.makedirs(export_path, exist_ok = True)

token_subfolder = 'tokens_dia_test'
subfolder = 'noise_dia_test'


ddim_sampler = DDIMSampler(model)

analogy_creator =  AnalogyCreator(config, ddim_sampler, subfolder, token_subfolder, os.path.join(log_config.experiment_root, 'analogy_results', out_subfolder))


Global seed set to 42


Loading model...
Loading model from stable-diffusion/models/ldm/stable-diffusion-v1/model.ckpt
Global Step: 194366
LatentDiffusion: Running in eps-prediction mode


# Dataset

In [ ]:
import ipyplot

im_path = lambda x: os.path.join(path_to_data, x)


all_images = sorted(utils.load_all_image_names())
labels = [utils.extract_file_id_from_path(x) for x in all_images] 


images_list = [im_path(x) for x in all_images]
ipyplot.plot_images(images_list, labels = labels, max_images=200, img_width=100)

### Triplet Selection

In [ ]:
file_id_A = '000203'
file_id_A_prime = '000204'
file_id_B = '000226'



# Analogies

### Check for inverted CLIP features and noise
The parameters will be estimated, if missing.

In [ ]:

# check for CLIP features
utils.check_and_run_inversion(model, file_id_A, token_subfolder, config, tokens=True)

utils.check_and_run_inversion(model,file_id_A_prime, token_subfolder, config, tokens=True)

utils.check_and_run_inversion(model,file_id_B, token_subfolder, config, tokens=True)


# check for noise of image B
config.token_subfolder = token_subfolder
utils.check_and_run_inversion(model, file_id_B, subfolder, config, tokens=False)

### Performing analogies
Generating grid of analogies for given triplet, list of guidance scales \sigma_i and analogy strengths \lambda_j.

In [ ]:



print(f'Processing triplet: ({file_id_A}, {file_id_A_prime}, {file_id_B})')

triplet = (utils.file_id2im_path(file_id_A), utils.file_id2im_path(file_id_A_prime), utils.file_id2im_path(file_id_B))
triplet_code = f'{file_id_A}_{file_id_A_prime}_{file_id_B}'

# cA = model.get_learned_conditioning('manual prompt for A')
# cAprime = model.get_learned_conditioning("manual prompt for A'")

cA = analogy_creator.fetch_cond_matrix(file_id_A)
cAprime = analogy_creator.fetch_cond_matrix(file_id_A_prime)

noiseB,_,_ = utils.load_inversion_result_dict(file_id_B, subfolder, return_result_dict=False)

cB = analogy_creator.fetch_cond_matrix(file_id_B)


analogy_creator.make_analogy_from_args(triplet_code, cA, cAprime, cB, noiseB, steps, scales)



### Analogy Results
Generated grid of analogies is shown. Each row corresponds to the guidance scale \sigma_i, each column corresponds to analogy strength \lambda_j.

In [ ]:
from IPython.display import Image
Image(filename=os.path.join(export_path,
                        'grids',
                        f'{triplet_code}_analogy_grid.jpg' )) 

#### Visualize particular result for given \sigma_i and \lambda_j.

In [ ]:
row = 3 # [0-len(scales)]
col = 10 # [0-len(steps)]
current_scale = scales[row]
current_step = steps[col]
print(f'scale: {current_scale} | step: {current_step}')
Image(filename=os.path.join(export_path,
                            triplet_code,
                        f'analogy_sc={current_scale}_shift_strength={current_step}.jpg' )) 

### Token visualization
Estimated CLIP features are paired with random noise images (and transformed via the reverse diffusion process) to visualize the captured concepts. 

In [ ]:

if visualize_tokens:
    from torchvision.utils import save_image
    os.makedirs(os.path.join(experiment_root,'token_visualization'),exist_ok=True)
    def gen_random_samples(cond, file_id):
        tokens_,_ = analogy_creator.ddim_sampler.sample(
                        analogy_creator.config.ddim_steps,
                        8,
                        analogy_creator.config.shape,
                        conditioning = cond.expand(8,-1,-1),
                        eta=analogy_creator.config.ddim_eta,
                        unconditional_guidance_scale=sc,
                        unconditional_conditioning=analogy_creator.uc.expand(8,-1,-1),
                    )
        utils.save_latent_as_image(
            analogy_creator.ddim_sampler.model, 
            tokens_,
            os.path.join(experiment_root,'token_visualization',f'{file_id}.jpg')
        )

    from IPython.display import Image
    gen_random_samples(cA, file_id_A)
    Image(filename=os.path.join(experiment_root,'token_visualization',f'{file_id_A}.jpg')) 
    gen_random_samples(cAprime, file_id_A_prime)
    Image(filename=os.path.join(experiment_root,'token_visualization',f'{file_id_A_prime}.jpg')) 
    gen_random_samples(cB, file_id_B)
    Image(filename=os.path.join(experiment_root,'token_visualization',f'{file_id_B}.jpg')) 

